In [1]:
# Install required libraries
# !pip install yfinance pandas numpy scipy matplotlib seaborn

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_nasdaq100_symbols():
    nasdaq100 = pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100')[4]
    return nasdaq100['Ticker'].tolist()

symbols = get_nasdaq100_symbols()

In [6]:
def get_stock_data(symbol):
    stock = yf.Ticker(symbol)
    
    try:
        info = stock.info
        financials = stock.financials
        balance_sheet = stock.balance_sheet
        
        # Enterprise Value
        ev = info.get('enterpriseValue', np.nan)
        
        # EBIT
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit', financials.columns[0]]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income', financials.columns[0]]
        else:
            revenue = financials.loc['Total Revenue', financials.columns[0]]
            cogs = financials.loc['Cost Of Revenue', financials.columns[0]] if 'Cost Of Revenue' in financials.index else 0
            operating_expenses = financials.loc['Total Operating Expenses', financials.columns[0]] if 'Total Operating Expenses' in financials.index else 0
            ebit = revenue - cogs - operating_expenses
        
        # Acquirer's Multiple
        acquirers_multiple = ev / ebit if ev and ebit and ebit != 0 else np.nan
        
        # Total Assets and Liabilities
        total_assets = balance_sheet.loc['Total Assets', balance_sheet.columns[0]]
        total_liabilities = balance_sheet.loc['Total Liabilities Net Minority Interest', balance_sheet.columns[0]] if 'Total Liabilities Net Minority Interest' in balance_sheet.index else np.nan
        current_liabilities = balance_sheet.loc['Total Current Liabilities', balance_sheet.columns[0]] if 'Total Current Liabilities' in balance_sheet.index else np.nan
        
        # Calculate Total Equity
        if 'Total Stockholder Equity' in balance_sheet.index:
            total_equity = balance_sheet.loc['Total Stockholder Equity', balance_sheet.columns[0]]
        elif not np.isnan(total_liabilities):
            total_equity = total_assets - total_liabilities
        else:
            total_equity = np.nan
        
        # ROIC
        invested_capital = total_assets - current_liabilities if not np.isnan(current_liabilities) else total_assets
        roic = ebit / invested_capital if invested_capital and invested_capital != 0 else np.nan
        
        # Revenue Growth
        if len(financials.columns) >= 4:
            revenue_recent = financials.loc['Total Revenue', financials.columns[0]]
            revenue_past = financials.loc['Total Revenue', financials.columns[3]]
            revenue_growth = (revenue_recent / revenue_past) ** (1/3) - 1 if revenue_past != 0 else np.nan
        else:
            revenue_growth = np.nan
        
        # Simplified Altman Z-Score
        retained_earnings = balance_sheet.loc['Retained Earnings', balance_sheet.columns[0]] if 'Retained Earnings' in balance_sheet.index else 0
        current_assets = balance_sheet.loc['Total Current Assets', balance_sheet.columns[0]] if 'Total Current Assets' in balance_sheet.index else 0
        working_capital = current_assets - current_liabilities if not np.isnan(current_liabilities) else 0
        z_score = (1.2 * working_capital / total_assets) + (1.4 * retained_earnings / total_assets) + (3.3 * ebit / total_assets) if total_assets != 0 else np.nan
        
        # Earnings Stability
        if len(financials.columns) >= 4:
            earnings = financials.loc['Net Income', :]
            earnings_stability = earnings.std() / earnings.mean() if earnings.mean() != 0 else np.nan
        else:
            earnings_stability = np.nan
        
        # Debt to Equity Ratio
        total_debt = balance_sheet.loc['Total Debt', balance_sheet.columns[0]] if 'Total Debt' in balance_sheet.index else np.nan
        if np.isnan(total_debt):
            long_term_debt = balance_sheet.loc['Long Term Debt', balance_sheet.columns[0]] if 'Long Term Debt' in balance_sheet.index else 0
            short_term_debt = balance_sheet.loc['Short Term Debt', balance_sheet.columns[0]] if 'Short Term Debt' in balance_sheet.index else 0
            total_debt = long_term_debt + short_term_debt
        
        debt_to_equity = total_debt / total_equity if total_equity != 0 and not np.isnan(total_equity) else np.nan
        
        beta = info.get('beta', np.nan)
        market_cap = info.get('marketCap', np.nan)
        avg_volume = info.get('averageVolume', np.nan)
        
        return pd.Series({
            'Symbol': symbol,
            'Acquirers_Multiple': acquirers_multiple,
            'ROIC': roic,
            'Revenue_Growth': revenue_growth,
            'Z_Score': z_score,
            'Earnings_Stability': earnings_stability,
            'Debt_to_Equity': debt_to_equity,
            'Beta': beta,
            'Market_Cap': market_cap,
            'Avg_Volume': avg_volume
        })
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        return pd.Series({
            'Symbol': symbol,
            'Acquirers_Multiple': np.nan,
            'ROIC': np.nan,
            'Revenue_Growth': np.nan,
            'Z_Score': np.nan,
            'Earnings_Stability': np.nan,
            'Debt_to_Equity': np.nan,
            'Beta': np.nan,
            'Market_Cap': np.nan,
            'Avg_Volume': np.nan
        })

In [7]:
# Fetch data for all symbols
data = pd.DataFrame([get_stock_data(symbol) for symbol in symbols])

# Save data as csv
data.to_csv('stock_data2.csv', index=False)

In [8]:
data=pd.read_csv('stock_data2.csv')
data.head()

,Symbol,Acquirers_Multiple,ROIC,Revenue_Growth,Z_Score,Earnings_Stability,Debt_to_Equity,Beta,Market_Cap,Avg_Volume
0,ADBE,35.057655,0.223312,0.146826,2.304624,0.064896,0.247003,1.298,235108417536,3294411
1,ABNB,41.847183,0.073529,0.431853,0.010385,9.048877,0.282180,1.189,72500969472,4624822
2,GOOGL,22.786645,0.209480,0.189751,1.426253,0.263049,0.100586,1.046,2004699971584,25150666
3,GOOG,23.051232,0.209480,0.189751,1.426253,0.263049,0.100586,1.046,2004697088000,17941091
4,AMZN,49.085938,0.069815,0.141867,0.531732,0.794664,0.671757,1.155,1740170461184,43470169


In [9]:
def calculate_score(data):
    # Calculate percentile ranks
    data['Value_Rank'] = data['Acquirers_Multiple'].rank(ascending=False, pct=True)
    data['Quality_Rank'] = (data['ROIC'].rank(pct=True) + (1 - data['Earnings_Stability'].rank(pct=True))) / 2
    data['Growth_Rank'] = data['Revenue_Growth'].rank(pct=True)
    data['Financial_Health_Rank'] = (data['Z_Score'].rank(pct=True) + (1 - data['Debt_to_Equity'].rank(pct=True))) / 2
    
    # Determine market conditions (simplified example)
    market_condition = 'bull'  # This should be determined based on actual market indicators
    
    # Set weights based on market conditions
    if market_condition == 'bull':
        weights = {'Value': 0.25, 'Quality': 0.2, 'Growth': 0.3, 'Financial_Health': 0.25}
    else:  # bear market
        weights = {'Value': 0.3, 'Quality': 0.25, 'Growth': 0.2, 'Financial_Health': 0.25}
    
    # Calculate composite score
    data['Composite_Score'] = (
        weights['Value'] * data['Value_Rank'] +
        weights['Quality'] * data['Quality_Rank'] +
        weights['Growth'] * data['Growth_Rank'] +
        weights['Financial_Health'] * data['Financial_Health_Rank']
    )
    
    # Calculate risk score
    data['Risk_Score'] = (data['Beta'].rank(pct=True) + data['Debt_to_Equity'].rank(pct=True)) / 2
    
    # Apply margin of safety
    data['Final_Score'] = data['Composite_Score'] * (1 - (data['Risk_Score'] * 0.5))
    
    return data

In [10]:
# Calculate scores
scored_data = calculate_score(data)

# Apply filters
min_market_cap = 1e9  # $1 billion
min_avg_volume = 100000  # 100,000 shares

filtered_data = scored_data[
    (scored_data['Market_Cap'] >= min_market_cap) &
    (scored_data['Avg_Volume'] >= min_avg_volume) &
    (scored_data['Beta'] <= 2)  # Exclude extremely volatile stocks
]

# Rank stocks
final_ranking = filtered_data.sort_values('Final_Score', ascending=False).reset_index(drop=True)

In [11]:
# Save to csv
from datetime import datetime
final_ranking.to_csv(f'final_ranking_{datetime.now().strftime("%Y%m%d")}.csv', index=False)

In [12]:
print(final_ranking[['Symbol', 'Final_Score', 'Composite_Score', 'Risk_Score']].head(10))

  Symbol  Final_Score  Composite_Score  Risk_Score
0    PDD     0.718374         0.771003    0.136521
1   BKNG     0.625330         0.785002    0.406808
2   VRTX     0.598741         0.625442    0.085383
3  GOOGL     0.556977         0.661499    0.316015
4   GOOG     0.552725         0.656449    0.316015
5    MAR     0.549742         0.718085    0.468868
6   CRWD     0.548201         0.717077    0.471012
7   LULU     0.545703         0.749273    0.543381
8   REGN     0.536571         0.566114    0.104369
9   ODFL     0.532505         0.602689    0.232903
